In [1]:
import sys
import pandas as pd
import os
import re
import glob
import shutil
import numpy as np
from pathlib import Path
import shutil

from IPython.core.display import display, HTML
# make the Jupyter notebook use the full screen width
display(HTML("<style>.container { width:99% !important; }</style>"))

In [30]:
!python ../../src/train.py \
--filename /notebook/nas-trainings/arne/DGFISMA/DATA/doc_classifier/DATA_DUMP_28_04/processed/train_data.tsv \
--output_dir  /notebook/nas-trainings/arne/DGFISMA/Doc_class-docker/code_baseline/models_dgfisma  \
--vectorizer_type tfidf  \
--feature_selection_svc 

/miniconda/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
7625 documents - 1.686MB (training set)
2 categories

Using Tfidf Vectorizer.
Extracting features via sklearn.feature_selection.SelectKbest using LinearSVC.


In [31]:
!python ../../src/test.py \
--filename /notebook/nas-trainings/arne/DGFISMA/DATA/doc_classifier/DATA_DUMP_28_04/processed/test_data.tsv  \
--model_path  /notebook/nas-trainings/arne/DGFISMA/Doc_class-docker/code_baseline/models_dgfisma/model.p \
--output_file /notebook/nas-trainings/arne/DGFISMA/Doc_class-docker/code_baseline/notebooks/results/results_tf_idf_binary_eurolex

/miniconda/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
2000 documents - 0.438MB (test set)
2 categories

              precision    recall  f1-score   support

           0       0.92      0.95      0.94      1503
           1       0.84      0.75      0.79       497

   micro avg       0.90      0.90      0.90      2000
   macro avg       0.88      0.85      0.86      2000
weighted avg       0.90      0.90      0.90      2000



In [32]:
!python ../../src/predict.py \
--filename /notebook/nas-trainings/arne/DGFISMA/DATA/doc_classifier/DATA_DUMP_28_04/processed/test_data_base64 \
--model_path  /notebook/nas-trainings/arne/DGFISMA/Doc_class-docker/code_baseline/models_dgfisma/model.p  \
--output_file /notebook/nas-trainings/arne/DGFISMA/Doc_class-docker/code_baseline/notebooks/results/results_tf_idf_binary_eurolex_testing

2000 documents - 0.438MB (test set)

/miniconda/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


### Some testing:

In [33]:
from base64 import b64encode
from base64 import b64decode

pred=open( "/notebook/nas-trainings/arne/DGFISMA/Doc_class-docker/code_baseline/notebooks/results/results_tf_idf_binary_eurolex_testing" ).read().rstrip( "\n" ).split( "\n" )
pred_probs=[ float( prediction.split()[2][:-1] ) for prediction in pred]  
pred_labels=[ int( prediction.split()[0] ) for prediction in pred]  

true_labels= open( "/notebook/nas-trainings/arne/DGFISMA/DATA/doc_classifier/DATA_DUMP_28_04/processed/test_data_labels" ).read().rstrip( "\n" ).split( "\n" )
true_labels=[ int(label) for label in true_labels  ]

sentences= open( "/notebook/nas-trainings/arne/DGFISMA/DATA/doc_classifier/DATA_DUMP_28_04/processed/test_data_base64" ).read().rstrip( "\n" ).split( "\n" )
sentences=[ b64decode( sentence ).decode()  for sentence in sentences  ]

In [34]:
from sklearn.metrics import classification_report
print( classification_report( true_labels, pred_labels ))

              precision    recall  f1-score   support

           0       0.92      0.95      0.94      1503
           1       0.84      0.75      0.79       497

   micro avg       0.90      0.90      0.90      2000
   macro avg       0.88      0.85      0.86      2000
weighted avg       0.90      0.90      0.90      2000



In [63]:
assert ( len( true_labels ) == len( pred_labels ) == len( pred_probs ) == len( sentences )   )

for true_label, pred_label, pred_prob, sentence in zip( true_labels, pred_labels, pred_probs , sentences  ):
    if true_label == pred_label:
        if pred_prob>0.9:
            print(  f"{sentence}, true_label: {true_label}, pred_label: {pred_label}, pred_prob: {pred_prob}" )
            print( "\n" )

2017 Council Regulation (EU) 2017/1836 of 10 October 2017 amending Regulation (EU) 2017/1509 concerning restrictive measures against the Democratic People's Republic of Korea, true_label: 1, pred_label: 1, pred_prob: 0.93195666


2017 Commission Delegated Decision (EU) 2017/2075 of 4 September 2017 replacing Annex VII to Directive 2012/34/EU of the European Parliament and of the Council establishing a single European railway area (Text with EEA relevance. ), true_label: 1, pred_label: 1, pred_prob: 0.96606827


2003 2003/211/CFSP: Council Decision 2003/211/CFSP of 24 February 2003 concerning the conclusion of the Agreement between the European Union and the North Atlantic Treaty Organisation on the Security of Information, true_label: 1, pred_label: 1, pred_prob: 0.96000431


2016 Commission Implementing Regulation (EU) 2016/1414 of 24 August 2016 approving the active substance cyantraniliprole, in accordance with Regulation (EC) No 1107/2009 of the European Parliament and of the Counc